In [1]:
# Importing all necessary libraries.
import pandas as pd
import numpy as np

from heavy_tail_observations import BothSideWeibullNoise, BothSideParetoNoise, BothSideFrechetNoise
from heavy_tail_regressions import catoni_lin_reg, mom_lin_reg, pro, reg_lin_reg, trunc_lin_reg, trunc_reg

from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt

In [33]:
# Using data from Apple's stock.
# df = pd.read_csv('./sandp500/individual_stocks_5yr/individual_stocks_5yr/AAPL_data.csv')
# df = pd.read_csv('./sandp500/individual_stocks_5yr/individual_stocks_5yr/AMZN_data.csv')
# df = pd.read_csv('./sandp500/individual_stocks_5yr/individual_stocks_5yr/NFLX_data.csv')
df = pd.read_csv('./sandp500/individual_stocks_5yr/individual_stocks_5yr/FB_data.csv')
future_days = 1000
df['prediction'] = df['close'].shift(-future_days)
df.head()

,date,open,high,low,close,volume,Name,prediction
0,2013-02-08,28.89,29.17,28.51,28.5450,37662614,FB,130.98
1,2013-02-11,28.61,28.68,28.04,28.2600,36979533,FB,130.32
2,2013-02-12,27.67,28.16,27.10,27.3700,93417215,FB,133.23
3,2013-02-13,27.36,28.32,27.31,27.9075,50100805,FB,130.84
4,2013-02-14,28.02,28.63,28.01,28.5000,35581045,FB,130.98


In [34]:
# X = df.drop(['date','Name','prediction'], 1).to_numpy()[:-future_days]
X = df['close'].to_numpy()[:-future_days]
X = np.concatenate([[X],[np.ones_like(X)]]).T
print(X.shape)
y = df['prediction'].to_numpy()[:-future_days]
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

(259, 2)


In [41]:
p = 1.5
optim_names = ['Catoni', 'MoM', 'PRo', 'Ridge', 'Trunc Lin', 'Trunc']
optim_params = [{'beta':1.},
                {'lam':1.,'k':10},
                {'lam':1.,'beta':100000.},
                {'lam':1.},
                {'lam':1.,'beta':100000.},
                {'lam':1.}]
optims = [catoni_lin_reg, mom_lin_reg, pro, reg_lin_reg, trunc_lin_reg, trunc_reg]

for opt_idx, (optim, optim_param, optim_name)  in enumerate(zip(optims,optim_params,optim_names)):
    print(opt_idx, optim_name, "optimization starts")
    y_est, w_hat = optim(*(x_train,y_train,x_test,p),**optim_param)
    error = np.mean(np.abs(y_est - y_test))
    print('{} RMS: {:.06f}'.format(optim_name, error))

0 Catoni optimization starts
Catoni RMS: 7.841653
1 MoM optimization starts
MoM RMS: 17.556416
2 PRo optimization starts
PRo RMS: 7.060685
3 Ridge optimization starts
Ridge RMS: 7.083085
4 Trunc Lin optimization starts
Trunc Lin RMS: 7.083085
5 Trunc optimization starts
Trunc RMS: 161.736523
